In [ ]:
# | default_exp _components.cli_docs_generator

In [ ]:
# | export

from typing import *

from nbdev.quarto import nbdev_readme
from nbdev.config import get_config
from nbdev.doclinks import NbdevLookup

from nbdev_mkdocs._helpers.doc_links_utils import fix_sym_links

In [ ]:

import shutil
from pathlib import Path
from unittest.mock import patch, MagicMock
from contextlib import contextmanager
from tempfile import TemporaryDirectory

from fastcore.foundation import Config

In [ ]:
@contextmanager
def mock_nbdev_readme(mock_contents, d):
    with patch('__main__.nbdev_readme') as mock_nbdev_readme:
        mock_nbdev_readme.__wrapped__ = MagicMock()
        with open((Path(d) / "README.md"), "w", encoding="utf-8") as f:
            f.write(mock_contents)
        yield
        
        

_mock_nbdev_readme_return_value = """This is a link to a symbol [`prepare`](https://airtai.github.io/nbdev-mkdocs/mkdocs.html#prepare)"""
with TemporaryDirectory() as d:
    with mock_nbdev_readme(_mock_nbdev_readme_return_value, d):
        nbdev_readme.__wrapped__()
        
        with open((Path(d) / "README.md"), "r", encoding="utf-8") as f:
            contents = f.read()
        
        print(contents)
        assert contents == _mock_nbdev_readme_return_value

This is a link to a symbol [`prepare`](https://airtai.github.io/nbdev-mkdocs/mkdocs.html#prepare)


In [ ]:
@contextmanager
def mock_get_config(d, cfg):
    with patch("__main__.get_config") as mock_get_config:
        mock_get_config.return_value = cfg
        yield

with TemporaryDirectory() as d:
    _d = dict(
        config_path=Path(d),
        doc_host="https://airtai.github.io",
        lib_path=Path(d) / "nbdev_mkdocs",
        doc_baseurl="/nbdev-mkdocs",
        docs_versioning="minor",
        version="0.7.0rc0",
        lib_name="nbdev-mkdocs",
    )

    _types = dict(config_path=Path, doc_host=str, lib_path=Path, doc_baseurl=str, docs_versioning=str, version=str, lib_name=str)
    cfg = Config(str(Path(d)), "test_settings.ini", create=_d, save=False, types=_types)
    with mock_get_config(d, cfg):
        cfg = get_config()

        print(cfg.config_path)
        assert cfg.config_path == Path(d)
        assert cfg.doc_host == 'https://airtai.github.io'
        assert cfg.lib_path.name == 'nbdev_mkdocs'

/tmp/tmpna2i1g2b


In [ ]:
# | export


def update_readme() -> None:
    """Update the readme file and fix the symbol links"""
    cfg = get_config()
    readme_path = cfg.config_path / "README.md"
    nbdev_readme.__wrapped__()

    with open(readme_path, "r", encoding="utf-8") as f:
        contents = f.read()

    contents = fix_sym_links(
        s=contents,
        nbdev_lookup=NbdevLookup(incl_libs=cfg.lib_name.replace("_", "-")),
        docs_versioning=cfg.get("docs_versioning", ""),
        lib_version=cfg.version,
        use_relative_doc_links=False,
        use_latest_doc_version=True,
    )

    with open(readme_path, "w", encoding="utf-8") as f:
        f.write(contents)

In [ ]:
fixture = """This is a link to a symbol [`prepare`](https://airtai.github.io/nbdev-mkdocs/mkdocs.html#prepare)"""
expected = {
    "minor": "This is a link to a symbol [`prepare`](https://airtai.github.io/nbdev-mkdocs/latest/api/nbdev_mkdocs/mkdocs/prepare/#nbdev_mkdocs.mkdocs.prepare)",
    "patch": "This is a link to a symbol [`prepare`](https://airtai.github.io/nbdev-mkdocs/latest/api/nbdev_mkdocs/mkdocs/prepare/#nbdev_mkdocs.mkdocs.prepare)",
    "None": "This is a link to a symbol [`prepare`](https://airtai.github.io/nbdev-mkdocs/api/nbdev_mkdocs/mkdocs/prepare/#nbdev_mkdocs.mkdocs.prepare)"
}

for docs_versioning in ["minor", "patch", "None", ""]:
    with TemporaryDirectory() as d:
        readme_path = Path(d) / "README.md"
        _d = dict(
            config_path=Path(d),
            doc_host="https://airtai.github.io",
            lib_path=Path(d) / "nbdev_mkdocs",
            doc_baseurl="/nbdev-mkdocs",
            docs_versioning=docs_versioning,
            version="0.1.0",
            lib_name="nbdev-mkdocs",
        )

        _types = dict(config_path=Path, doc_host=str, lib_path=Path, doc_baseurl=str, docs_versioning=str, version=str, lib_name=str)
        cfg = Config(str(Path(d)), "test_settings.ini", create=_d, save=False, types=_types)
        with mock_get_config(d, cfg):
            with mock_nbdev_readme(fixture, d):
                update_readme()

            with open(readme_path, "r", encoding="utf-8") as f:
                actual = f.read()
            print(actual)
            if docs_versioning == "":
                assert actual == expected["None"]
            else:
                assert actual == expected[docs_versioning], f"actual={actual}, expected={expected[doc_version]}"

This is a link to a symbol [`prepare`](https://airtai.github.io/nbdev-mkdocs/latest/api/nbdev_mkdocs/mkdocs/prepare/#nbdev_mkdocs.mkdocs.prepare)
This is a link to a symbol [`prepare`](https://airtai.github.io/nbdev-mkdocs/latest/api/nbdev_mkdocs/mkdocs/prepare/#nbdev_mkdocs.mkdocs.prepare)
This is a link to a symbol [`prepare`](https://airtai.github.io/nbdev-mkdocs/api/nbdev_mkdocs/mkdocs/prepare/#nbdev_mkdocs.mkdocs.prepare)
This is a link to a symbol [`prepare`](https://airtai.github.io/nbdev-mkdocs/api/nbdev_mkdocs/mkdocs/prepare/#nbdev_mkdocs.mkdocs.prepare)


In [ ]:
fixture = """This is a link to a symbol [`prepare`](https://airtai.github.io/nbdev-mkdocs/mkdocs.html#prepare)"""
expected = "This is a link to a symbol [`prepare`](https://airtai.github.io/nbdev-mkdocs/api/nbdev_mkdocs/mkdocs/prepare/#nbdev_mkdocs.mkdocs.prepare)"

with TemporaryDirectory() as d:
    readme_path = Path(d) / "README.md"
    _d = dict(
        config_path=Path(d),
        doc_host="https://airtai.github.io",
        lib_path=Path(d) / "nbdev_mkdocs",
        doc_baseurl="/nbdev-mkdocs",
        version="0.1.0",
        lib_name="nbdev-mkdocs",
    )

    _types = dict(config_path=Path, doc_host=str, lib_path=Path, doc_baseurl=str, version=str, lib_name=str)
    cfg = Config(str(Path(d)), "test_settings.ini", create=_d, save=False, types=_types)
    with mock_get_config(d, cfg):
        with mock_nbdev_readme(fixture, d):
            update_readme()

        with open(readme_path, "r", encoding="utf-8") as f:
            actual = f.read()
        print(actual)
        assert actual == expected

This is a link to a symbol [`prepare`](https://airtai.github.io/nbdev-mkdocs/api/nbdev_mkdocs/mkdocs/prepare/#nbdev_mkdocs.mkdocs.prepare)
